In [1]:
# All library for CNN Model
# Using Tenserflow backend

import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
import tensorflow as tf
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# Color pilex size
Image_Size = [224,224]

# Setting up the path
train_path = 'C:/Symbiosis/Summer_Project/chest_xray/chest_xray/train'
test_path = 'C:/Symbiosis/Summer_Project/chest_xray/chest_xray/test'

In [3]:
# Use to predict our model
vgg = VGG16(input_shape = Image_Size + [3], weights = 'imagenet', include_top = False)

#For ignoring the untrain file
for layer in vgg.layers:
    layer.trainable = False

In [4]:
#Output length
folders = glob('C:/Symbiosis/Summer_Project/chest_xray/chest_xray/train/*')
#Adding more layer
x = Flatten()(vgg.output)

In [5]:
prediction = Dense(len(folders), activation='softmax')(x)

# Create the model project
model = Model(inputs=vgg.input, outputs=prediction)

# View the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
#Compile and optimization
model.compile(
   loss = 'categorical_crossentropy',
   optimizer = 'adam',
   metrics=['accuracy']
)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Provide same target size as initialied for the image size

training_set = train_datagen.flow_from_directory('C:/Symbiosis/Summer_Project/chest_xray/chest_xray/train',
                                                target_size=(224,224),
                                                batch_size=10,
                                                class_mode='categorical')


test_set = test_datagen.flow_from_directory('C:/Symbiosis/Summer_Project/chest_xray/chest_xray/test',
                                           target_size=(224,224),
                                           batch_size=10,
                                           class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [8]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set))

522/522 [==============================] - 517s 989ms/step - loss: 0.1900 - accuracy: 0.9285 - val_loss: 0.4470 - val_accuracy: 0.8862


In [9]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')

from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [10]:
model = load_model('chest_xray.h5')

In [19]:
img = image.load_img('C:/Symbiosis/Summer_Project/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg',target_size=(224,224))

In [20]:
x = image.img_to_array(img)

In [21]:
x = np.expand_dims(x, axis=0)

In [22]:
img_data = preprocess_input(x)

In [23]:
classes = model.predict(img_data)

In [24]:
#classes
result = classes[0][0]

In [25]:
if result>0.5:
    print("Result is Normal")
else:
    print("Person is effected from PNEUMONIA")

Person is effected from PNEUMONIA
